In [35]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver 
from bs4 import BeautifulSoup 
import urllib.request as req 
import pandas as pd 
import numpy as np 
import collections as co



In [22]:
from selenium import webdriver 
url = "https://play.google.com/store/apps/details?id=gov.busan.dongbaekpay&hl=ko&gl=US&showAllReviews=true" # 접속하고자하는 url 
driverPath = "D:/chromedriver/chromedriver.exe" # Chrome Driver path 
driver = webdriver.Chrome(driverPath) # Open Chrome 
driver.get(url) # Enter the url



- 스크롤 내리기

In [8]:
import time
SCROLL_PAUSE_TIME = 1.5 
last_height = driver.execute_script("return document.body.scrollHeight") 
while True: 
    # (1) 4번의 스크롤링 
    for i in range(4): 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(SCROLL_PAUSE_TIME)
    # (2) 더보기 클릭 
    try:
        driver.find_element_by_xpath("//span[@class='RveJvd snByac']").click()
    except:
        pass
    # (3) 종료 조건 
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height: 
        break
    last_height = new_height
    
    
     
    
    



In [20]:
## [전체 리뷰] 버튼 클릭하여 펼치기 
spread_review = driver.find_elements_by_xpath("//button[@jsaction='click:TiglPc']") 
for i in range(len(spread_review)): 
    isTrue = spread_review[i].is_displayed() 
    print("Element is visible? " + str(isTrue)) 
    if isTrue: 
        spread_review[i].click() 
        print(str(i)+"th more button is clicked and wait 1.5 secs...") 
        time.sleep(SCROLL_PAUSE_TIME)



WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.107)


In [21]:
reviews = driver.find_elements_by_xpath("//span[contains(@jsname, 'bN97Pc')]") 
# for i in range(len(reviews)): 
#     print(str(i) + "\t" + reviews[i].text) 
long_reviews = driver.find_elements_by_xpath("//span[@jsname='fbQN7e']") 
# for i in range(len(reviews)): 
#     print(str(i) + "\t" + long_reviews[i].text)




WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.107)


In [12]:
empty = [i for i,t in enumerate(reviews) if t.text==''] 
long = [i for i,t in enumerate(long_reviews) if t.text!=''] 
empty == long



True

In [13]:
# merge two list 
merged_review = [t.text if t.text!='' else long_reviews[i].text for i,t in enumerate(reviews)] 



In [17]:
dates = driver.find_elements_by_xpath("//div[@class='bAhLNe kx8XBd']/div/span[@class='p2TkOb']") 
likes = driver.find_elements_by_xpath("//div[@aria-label='이 리뷰가 유용하다는 평가를 받은 횟수입니다.']") 
stars = driver.find_elements_by_xpath("//span[@class='nt2C1d']/div[@class='pf5lIe']/div[@role='img']") 
stars_text = stars[3].get_attribute('aria-label')



In [18]:
dates

[<selenium.webdriver.remote.webelement.WebElement (session="314d35d62d47e77b1d44afbe2fe033dc", element="7555e797-89b2-4679-b801-88b15a98221f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="314d35d62d47e77b1d44afbe2fe033dc", element="efbcecb6-de11-4fde-ab13-8605fbaf71be")>,
 <selenium.webdriver.remote.webelement.WebElement (session="314d35d62d47e77b1d44afbe2fe033dc", element="dfae64ad-f5f8-48bb-abc7-5d87a0f26c97")>,
 <selenium.webdriver.remote.webelement.WebElement (session="314d35d62d47e77b1d44afbe2fe033dc", element="103f424f-8dc2-4b91-90b9-f5f7dd6ed0a4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="314d35d62d47e77b1d44afbe2fe033dc", element="0f8f6dc7-2cdc-430f-aa43-92d00068ee77")>,
 <selenium.webdriver.remote.webelement.WebElement (session="314d35d62d47e77b1d44afbe2fe033dc", element="a5f3acb0-ae71-41b2-8d33-3a668f5b9e86")>,
 <selenium.webdriver.remote.webelement.WebElement (session="314d35d62d47e77b1d44afbe2fe033dc", element="c030e6b8-61d5-41c9-a555-a3

In [19]:
res_deque = co.deque([]) 
for i in range(len(reviews)): 
    res_deque.append({ 
        'DATE' : dates[i].text, 
        'STAR' : stars[i].get_attribute('aria-label'), 
        'LIKE' : likes[i].text, 
        'REVIEW' : merged_review[i] 
    }) 
res_df = pd.DataFrame(res_deque) 
res_df.to_csv('D:/Crawling_data.csv', index=False, encoding='utf-8-sig')



In [19]:
res_df = pd.read_csv("D:/4월이후data.csv")
res_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    860 non-null    object 
 1   STAR    860 non-null    object 
 2   LIKE    443 non-null    float64
 3   REVIEW  860 non-null    object 
dtypes: float64(1), object(3)
memory usage: 27.0+ KB
